# Model Fine-Tuning and Export

## Preamble

In [ ]:
import os

KAGGLE_ENVIRONMENT = False

if os.path.isdir("/kaggle/working"):
    KAGGLE_ENVIRONMENT = True
    
    import subprocess
    
    print("In Kaggle environment")
    
    # If we don't uninstall `wandb`, `ultralytics` will keep asking for an API key.
    print("Uninstalling `wandb`...")
    completed = subprocess.run(["pip", "uninstall", "wandb", "-y"], capture_output=True)
    if completed.returncode == 0:
        print("Successfully uninstalled `wandb`")
    else:
        print(f"Failed to uninstall `wandb`: {completed.stderr}")
        
    for pkg in ("roboflow", "fiftyone", "ultralytics"):
        print(f"Installing `{pkg}`...")
        completed = subprocess.run(["pip", "install", pkg], capture_output=True)
        if completed.returncode == 0:
            print(f"Successfully installed `{pkg}`")
        else:
            print(f"Failed to install `{pkg}`: {completed.stderr}")

### Imports

In [ ]:
import os
from pathlib import Path
import shutil
import zipfile
from collections import Counter
from typing import Optional

import cv2
import fiftyone as fo
import fiftyone.utils.random as four
import matplotlib.pyplot as plt
import pymongo
import yaml
from ray import tune
from roboflow import Roboflow
from roboflow.core.dataset import Dataset as RoboflowDataset
from ultralytics import YOLO

### General Configuration

In [ ]:
# Define folder paths.
ROOT_PATH = "/kaggle/working/root" if KAGGLE_ENVIRONMENT else os.path.abspath(os.getcwd())

IMAGES_PATH = os.path.join(ROOT_PATH, 'images')

# Name of the directory for combining datasets.
COMBINED_DATASET_DIR = "combined"

# Format to use when downloading Roboflow datasets.
RF_DATASET_FORMAT = "yolov8"

# Format to use when downloading FiftyOne datasets.
FO_DATASET_FORMAT = fo.types.YOLOv5Dataset # YOLOv5 and YOLOv8 use the same format

# Output path when exporting the model.
MODEL_OUTPUT_PATH = os.path.join(ROOT_PATH, "sentinel_default_v2.pt") # changed the model name for v2

FiftyOne uses MongoDB to manage its datasets. When possible, FiftyOne will automatically set up the database for you. However, when it fails to do so, you need to manually set up a MongoDB database. The code below checks if FiftyOne is able to set up the database — if not, then you must set up your own and specify the connection string. After installing MongoDB, run `mongod --dbpath <DBPATH>`, replacing `DBPATH` with any path of your choice. By default (no authentication and using the default port), the connection string is: `mongodb://localhost:27017`.

In [ ]:
while True:
    try:
        print("Trying to reach MongoDB...")
        fo.core.odm.database.get_db_config()
        print("MongoDB is reachable.")
        break
    except (fo.core.config.FiftyOneConfigError, pymongo.errors.ServerSelectionTimeoutError):
        print("Failed to reach a running MongoDB instance. Enter a valid MongoDB connection string:")
        db_uri = input()
        fo.config.database_uri = db_uri

## Datasets

Helper function to `gitignore` a directory:

In [ ]:
def gitignore(directory: str):
    """
    Make the given directory ignored by Git.

    No prefixes are prepended to the directory. The directory must already exist.

    This function adds a `.gitignore` file to the directory
    containing the wildcard pattern "*" so that git ignores it.
    """
    if not os.path.isdir(directory):
        raise ValueError("The given path does not exist or is not a directory.")
        
    gitignore_path = os.path.join(directory, ".gitignore")
    with open(gitignore_path, "w") as gitignore_file:
        gitignore_file.write("*")

We'll create the `IMAGES_PATH` directory early to make `git` ignore it:

In [ ]:
if not os.path.exists(IMAGES_PATH):
    os.makedirs(IMAGES_PATH)
    print(f"Created '{IMAGES_PATH}' directory.")
else:
    print(f"'{IMAGES_PATH}' exists — nothing to do.")

In [ ]:
if not os.path.exists(os.path.join(IMAGES_PATH, ".gitignore")):
    gitignore(IMAGES_PATH)
    print(f"Gitignored '{IMAGES_PATH}'.")
else:
    print(f"'{IMAGES_PATH}/.gitignore' exists — skipping.")

### Roboflow Datasets

To download datasets from Roboflow, you must have a Roboflow API key. This notebook will attempt to load the API key from the `ROBOFLOW_API_KEY` environment variable or Kaggle's secrets management. If the variable does not exist, then you will be prompted for it.

In [ ]:
rf_api_key: str | None = None

if "ROBOFLOW_API_KEY" in os.environ:
    rf_api_key = os.environ["ROBOFLOW_API_KEY"]
elif KAGGLE_ENVIRONMENT:
    try:
        from kaggle_secrets import UserSecretsClient
        kaggle_user_secrets = UserSecretsClient()
        rf_api_key = kaggle_user_secrets.get_secret("ROBOFLOW_API_KEY")
    except Exception as ex:
        print(f"Failed to retrieve Roboflow API key from Kaggle: {repr(ex)}")

if rf_api_key is None:
    print("Could not find Roboflow API key.")
    print("Please enter your Roboflow API key: ")
    rf_api_key = input()

rf = Roboflow(api_key=rf_api_key)

In [ ]:
def download_roboflow_dataset(workspace: str, project: str, version: str, directory: str, dataset_format=RF_DATASET_FORMAT):
    """
    Downloads the specified Roboflow dataset into the given directory
    and returns the dataset as a Roboflow `Dataset` object.

    The directory will be prefixed by `IMAGES_PATH`.

    If the directory already exists, the dataset will not be redownloaded.
    """
    abs_directory = os.path.join(IMAGES_PATH, directory)

    rf_project = rf.workspace(workspace).project(project)
    rf_version = rf_project.version(version)
    
    if os.path.exists(abs_directory):
        print(f"Path '{abs_directory}' exists — refusing to overwrite.")
        print("If you want to redownload the dataset, please manually remove the directory.")
        return RoboflowDataset(rf_version.name, rf_version.version, dataset_format, abs_directory)
        
    dataset = rf_version.download(dataset_format, location=abs_directory)

    print(f"Dataset downloaded to: {abs_directory}")
    
    return dataset

In [ ]:
gun_ds = download_roboflow_dataset("liteye-systems", "weapon-classification", "2", "guns")
knife_ds = download_roboflow_dataset("knife-detection-sjzqp", "knife-detection-bstjz", "2", "knife") # new knife dataset
parcel_ds = download_roboflow_dataset("king-mongkuts-institute-of-technology-ladkrabang-vaztb", "package-detection-hfpr9", "4", "parcel") # new parcel dataset

Unfortunately, due to a [Roboflow bug](https://github.com/roboflow/roboflow-python/issues/240), the paths in the YAML file are wrong, so we'll manually fix them:

In [ ]:
def fix_dataset_yaml_paths(yaml_path: str, train_rel_path: str, valid_rel_path: str, test_rel_path: str):
    with open(yaml_path, 'r') as file:
        yaml_content = yaml.safe_load(file)

    yaml_content["train"] = train_rel_path
    yaml_content["val"] = valid_rel_path
    yaml_content["test"] = test_rel_path

    with open(yaml_path, 'w') as file:
        yaml.dump(yaml_content, file)
        print(f"Updated '{yaml_path}'")

In [ ]:
fix_dataset_yaml_paths(
    os.path.join(IMAGES_PATH, "guns", "data.yaml"),
    train_rel_path="./train/images",
    valid_rel_path="./valid/images",
    test_rel_path="./test/images",
)

# New knife dataset
fix_dataset_yaml_paths(
    os.path.join(IMAGES_PATH, "knife", "data.yaml"),
    train_rel_path="./train/images",
    valid_rel_path="./valid/images",
    test_rel_path="./test/images",
)

# New parcel dataset 
fix_dataset_yaml_paths(
    os.path.join(IMAGES_PATH, "parcel", "data.yaml"),
    train_rel_path="./train/images",
    valid_rel_path="./valid/images",
    test_rel_path="./test/images",
)

### COCO Dataset

In [ ]:
def download_coco2017(
    categories: Optional[list[str]] = ["person"],
    max_samples: Optional[int] = None,
    directory: str = "coco-2017",
    dataset_format=FO_DATASET_FORMAT,
    seed: int = 0,
    **kwargs,
):
    """
    Downloads the COCO 2017 dataset into the given directory.

    All splits will be downloaded. The dataset can be filtered by category
    using the `categories` argument. If `max_samples` is specified, then each
    split will be limited to have a maximum of `max_samples` number of samples.
    
    By default, the dataset will be exported in the format specified by `FO_DATASET_FORMAT`.
    To change the output format, specify the `dataset_format` argument.
    """
    # Unfortunately, the test split of COCO 2017 does not have labels,
    # which makes it not so useful here.
    splits = ["train", "validation"]
    
    dataset = fo.zoo.load_zoo_dataset(
        "coco-2017",
        splits=splits,
        label_types=["detections"],
        max_samples=max_samples,
        **kwargs
    )

    # Rename 'validation' split to 'val'
    validation_view = dataset.match_tags("validation")
    validation_view.tag_samples("val")
    validation_view.untag_samples("validation")

    splits.remove("validation")
    splits.append("val")

    # Use half of the validation set as a test set.
    # Note that we are not explicitly tagging the test samples;
    # otherwise, multiple calls to this function will repeatedly shrink the validation set.
    validation_view = dataset.match_tags("val")
    validation_view, test_view = four.random_split(validation_view, [0.5, 0.5], seed=seed)
    train_view = dataset.match_tags("train")

    print(train_view.stats())
    print(validation_view.stats())
    print(test_view.stats())

    ds_view = dataset.view()

    # Manually filter the dataset to samples matching the given catgories
    # due to a bug: https://github.com/voxel51/fiftyone/issues/4570
    # Workaround based on: https://github.com/voxel51/fiftyone/issues/4570#issuecomment-2392548410
    # Unfortunately, the workaround downloads images we don't need and then filters them,
    # so we waste a bit of space and network bandwidth.
    if categories is not None:
        ds_view = ds_view.filter_labels("ground_truth", fo.ViewField("label").is_in(categories))

    # Export in YOLOv8 format.
    # According to https://github.com/voxel51/fiftyone/issues/3392#issuecomment-1666520356,
    # splits must be exported separately.
    export_dir = os.path.join(IMAGES_PATH, directory)
    for split, view in {
        "train": train_view,
        "val": validation_view,
        "test": test_view
    }.items():
        view.export(
            export_dir=export_dir,
            dataset_type=dataset_format,
            split=split,
            classes=categories,
        )
        print(f"Split '{split}' exported to '{export_dir}/{split}'")

In [ ]:
download_coco2017(max_samples=3000)

### Combining the Datasets

#### Update Labels

Before we can combine the datasets, we first need to update the labels for each dataset to use global indices. Otherwise, different datasets will use the same index for different classes. We'll first construct a map to keep track of the mapping between indices:

In [ ]:
def construct_index_map(dirs: list[tuple[str, str]], images_path: str = IMAGES_PATH) -> dict[str, dict[int | str, int]]:
    """
    Constructs a dictionary that maps the index and name of each category of the given datasets
    to a global index in preparation for combining them.
    """
    index = 0
    index_map: dict[str, dict[int, int]] = {}
    for ds_dir, ds_yaml in dirs:
        yaml_path = os.path.join(images_path, ds_dir, ds_yaml)
        index_map[ds_dir] = {}
        
        with open(yaml_path, "r") as file:
            yaml_content = yaml.safe_load(file)
            
        names = yaml_content["names"]
        if type(names) == dict:
            it = names.items()
        elif type(names) == list:
            it = enumerate(names)
        else:
            raise ValueError("Unknown type for 'names'.")

        for idx, name in it:
            index_map[ds_dir][idx] = index
            index_map[ds_dir][name] = index
            index += 1
            
    return index_map

In [ ]:
index_map = construct_index_map([
    ("coco-2017", "dataset.yaml"),
    ("guns", "data.yaml"),
    ("knife", "data.yaml"), # new knife dataset
    ("parcel", "data.yaml"), # new parcel dataset 
])
index_map

Now, we update the labels in each dataset's YAML file and `labels` directory (or directories):

In [ ]:
def _update_labels_yaml(
    ds_dir: str,
    yaml_rel_path: str,
    yaml_content: str,
    index_map: dict[str, dict[int, int]],
    images_path: str = IMAGES_PATH
):
    """
    Updates the labels in the given dataset's YAML file to use the global indices in `index_map`.

    Do not call this function directly! Call `update_labels()` instead to make sure the dataset remains consistent.
    """
    names = yaml_content["names"]
    if type(names) == dict:
        name_strings = names.values()
    elif type(names) == list:
        name_strings = names
    else:
        raise ValueError("Unknown type for 'names'.")
        
    new_yaml_names = {}
    for name in name_strings:
        index = index_map[ds_dir][name]
        new_yaml_names[index] = name

    yaml_content["names"] = new_yaml_names

    yaml_path = os.path.join(IMAGES_PATH, ds_dir, yaml_rel_path)
    with open(yaml_path, "w") as yaml_file:
        yaml.dump(yaml_content, yaml_file)

    print(f"Updated '{yaml_path}'")

In [ ]:
def _update_labels_txt(
    ds_dir: str,
    yaml_rel_path: str,
    yaml_content: str,
    index_map: dict[str, dict[int, int]],
    images_path: str = IMAGES_PATH,
):
    """
    Updates the labels in the given dataset's label files to use the global indices in `index_map`.
    
    Do not call this function directly! Call `update_labels()` instead to make sure the dataset remains consistent.
    """
    yaml_path = os.path.join(IMAGES_PATH, ds_dir, yaml_rel_path)
    for unresolved_img_dir in (yaml_content["train"], yaml_content["val"], yaml_content["test"]):
        if os.path.isabs(unresolved_img_dir):
            # Just use the directory as is.
            img_dir = unresolved_img_dir
        else: # Relative path
            parent_yaml_path = os.path.dirname(yaml_path) # Get parent directory of the YAML file.
            img_dir = os.path.join(parent_yaml_path, unresolved_img_dir) # Append the directory to the parent.
            img_dir = os.path.abspath(img_dir) # Normalise.

        img_dir = Path(img_dir)

        # Find the last occurrence of "images" in the path.
        # The way the YOLO format works to find the labels directory is to replace
        # the last occurrence of "images" in the path with "labels".
        img_dir_parts = list(img_dir.parts)
        last_images_idx = len(img_dir_parts) - 1 - img_dir_parts[::-1].index("images")
        assert img_dir.parts[last_images_idx] == "images"

        # Replace the last "images" with "labels" and reconstruct the path.
        img_dir_parts[last_images_idx] = "labels"
        labels_dir = Path(*img_dir_parts)

        # Iterate over the files in the labels directory.
        for label_file_path in labels_dir.iterdir():
            if not label_file_path.is_file():
                print(f"'{label_file_path}' is not a file — skipping")
                continue

            # Read label file contents.
            with label_file_path.open("r") as label_file:
                label_contents = label_file.readlines()

            # Modify label file class IDs
            with open(label_file_path, "w") as label_file:
                for line in label_contents:
                    parts = line.strip().split()
                    class_id = int(parts[0])
                    new_class_id = index_map[ds_dir][class_id]
                    new_line = f"{new_class_id} " + " ".join(parts[1:]) + "\n"
                    label_file.write(new_line)

        print(f"Updated labels in '{labels_dir}'")

In [ ]:
def update_labels(
    ds_dir: str,
    yaml_rel_path: str,
    index_map: dict[str, dict[int, int]],
    images_path: str = IMAGES_PATH,
):
    """
    Updates the labels in the given dataset to use the global indices in `index_map`.

    If this function raises an exception, then the dataset is likely corrupt!
    """
    if ds_dir not in index_map:
        raise ValueError(f"'{ds_dir}' is not in the given index map.")
    
    yaml_path = os.path.join(IMAGES_PATH, ds_dir, yaml_rel_path)
    with open(yaml_path, "r") as yaml_file:
        yaml_content = yaml.safe_load(yaml_file)

    # Check if the labels have already been updated.
    names = yaml_content["names"]
    if type(names) == dict:
        if not names:
            raise ValueError("No classes declared in dataset.")

        items = iter(names.items())
        first_ident, first_name = next(items)
        is_updated: bool = index_map[ds_dir][first_name] == first_ident

        # Check the remaining identifiers and names for consistency.
        for ident, name in items:
            if (index_map[ds_dir][name] == ident) != is_updated:
                raise ValueError("Detected partially updated dataset labels. The dataset is probably corrupt.")
    else:
        # If the labels had been updated, the type of `names` would have been `dict`.
        is_updated = False

    if is_updated:
        print("Dataset labels already updated — skipping")
    else:
        _update_labels_yaml(ds_dir, yaml_rel_path, yaml_content, index_map, images_path)
        _update_labels_txt(ds_dir, yaml_rel_path, yaml_content, index_map, images_path)

In [ ]:
update_labels("coco-2017", "dataset.yaml", index_map)

In [ ]:
update_labels("guns", "data.yaml", index_map)

In [ ]:
update_labels("knife", "data.yaml", index_map)

In [ ]:
update_labels("parcel", "data.yaml", index_map)

#### Create YAML File

Now that the labels have been updated, we can create a YAML file for the combined dataset:

In [ ]:
def create_combined_dataset_yaml(
    output_ds_dir: str,
    ds_yamls: list[tuple[str, str]],
    index_map: dict[str, dict[int, int]],
    images_path: str = IMAGES_PATH
):
    # Check if a YAML file already exists.
    output_ds_dir_abs = os.path.join(images_path, output_ds_dir)
    output_yaml_path = os.path.join(output_ds_dir_abs, "dataset.yaml")
    if os.path.isfile(output_yaml_path):
        print(f"'{output_yaml_path}' exists — refusing to overwrite")
        return
    
    combined_yaml_content = {
        "train": [],
        "val": [],
        "test": [],
    }
    
    combined_yaml_content["names"] = {
        idx: name
        for ds_index_map in index_map.values()
        for name, idx in ds_index_map.items()
        if type(name) == str # If not, then `name` refers to an old index, which we don't need here.
    }
    
    combined_yaml_content["nc"] = len(combined_yaml_content["names"])

    # Iterate the individual datasets.
    for ds_dir, yaml_rel_path in ds_yamls:
        yaml_path = os.path.join(images_path, ds_dir, yaml_rel_path)
        with open(yaml_path, "r") as file:
            yaml_content = yaml.safe_load(file)

        for split in ("train", "val", "test"):
            # In case the dataset is missing one of the splits.
            if split not in yaml_content:
                print(f"Warning: dataset '{ds_dir}' does not contain '{split}' split — skipping split")
                continue

            if type(yaml_content[split]) == str:
                split_rel_paths = [yaml_content[split]]
            elif type(yaml_content[split]) == list:
                split_rel_paths = yaml_content[split]
            else:
                raise ValueError(f"Encountered type '{type(yaml_content[split])}' for '{split}' field in '{yaml_path}' — don't know how to handle")

            # Parent directory of the YAML file, relative to `images_path`.
            parent_yaml_path = os.path.dirname(os.path.join(ds_dir, yaml_rel_path))
            split_paths = []
            for split_rel_path in split_rel_paths:
                # Path to the split, relative to `images_path`.
                split_path = os.path.join(parent_yaml_path, split_rel_path)

                # Path to the split, relative to the output directory
                # Assumes the output directory will be directly under `images_path`.
                split_path = os.path.join("..", split_path)
                split_paths.append(os.path.normpath(split_path))
            
            combined_yaml_content[split].extend(split_paths)

    
    # Create the output directory.
    if not os.path.isdir(output_ds_dir_abs):
        os.mkdir(output_ds_dir_abs)
        
    # Write combined YAML content out.
    with open(output_yaml_path, "w") as file:
        yaml.dump(combined_yaml_content, file)

    print(f"Successfully wrote dataset configuration to '{output_yaml_path}'")

In [ ]:
create_combined_dataset_yaml(
    COMBINED_DATASET_DIR,
    [
        ("coco-2017", "dataset.yaml"),
        ("guns", "data.yaml"),
        ("knife", "data.yaml"),
        ("parcel", "data.yaml"),
    ],
    index_map
)

# Hyperparameter Tuning

We'll perform hyperparameter tuning using Ultralytics' integration with Ray Tune. As part of this hyperparameter tuning, we also explore techniques and parameters for data augmentation.

In [ ]:
model = YOLO('yolov8n.pt')

In [ ]:
space = {
    "lr0": tune.uniform(1e-5, 1e-1),
    "lrf": tune.uniform(0.01, 1.0),
    "momentum": tune.uniform(0.6, 0.98),
    "weight_decay": tune.uniform(0.0, 0.001),
    "warmup_epochs": tune.uniform(0.0, 5.0),
    "warmup_momentum": tune.uniform(0.0, 0.95),
    "box": tune.uniform(0.02, 0.2),
    "cls": tune.uniform(0.2, 4.0),
    "hsv_h": tune.uniform(0.0, 0.2),
    "hsv_s": tune.uniform(0.0, 0.9),
    "hsv_v": tune.uniform(0.0, 0.9),
    "degrees": tune.uniform(0.0, 90.0),
    "translate": tune.uniform(0.0, 0.9),
    "scale": tune.uniform(0.0, 0.9),
    "shear": tune.uniform(0.0, 10.0),
    "perspective": tune.uniform(0.0, 0.001),
    "flipud": tune.uniform(0.0, 1.0),
    "fliplr": tune.uniform(0.0, 1.0),
    "bgr": tune.uniform(0.0, 1.0),
    "mosaic": tune.uniform(0.0, 1.0),
    "mixup": tune.uniform(0.0, 1.0),
    "copy_paste": tune.uniform(0.0, 1.0),
}

In [ ]:
result_grid = model.tune(
    data=os.path.join(IMAGES_PATH, COMBINED_DATASET_DIR, "dataset.yaml"),
    use_ray=True,
    space=space,
    epochs=25,
    grace_period=10,
    gpu_per_trial=2, # Tweak according to how many GPUs you have.
)

In [ ]:
from ultralytics.cfg import TASK2METRIC

metric = TASK2METRIC["detect"]
print(f"Retrieving best result with highest '{metric}'")

best_result = result_grid.get_best_result(metric=metric, mode="max")

In [ ]:
best_result.metrics

In [ ]:
best_result.config

# Fine-Tuning

Finally, we can fine-tune the model using the best set of hyperparameters:

In [ ]:
def fine_tune(model, yaml_path, epochs=5, imgsz=640, batch=16, device=None, patience=5, **train_kwargs):
    # Prepare the arguments for model.train().
    default_train_kwargs = {
        'data': yaml_path,
        'epochs': epochs, 
        'imgsz': imgsz,
        'batch': batch,
        'patience': patience,
    }

    # Include `device` only if it is specified.
    if device is not None: 
        train_kwargs['device'] = device
        
    # Merge other arguments.
    train_kwargs = default_train_kwargs | train_kwargs

    print(f"Train arguments: {train_kwargs}\n")

    model.train(**train_kwargs)
    return model

Use the hyperparameters determined by hyperparameter tuning:

In [ ]:
# TODO: replace with dictionary of best parameters from hyperparameter tuning (`best_result.config`).
train_kwargs = best_result.config.copy()

In [ ]:
# Remove values that shouldn't be in there when passing to the fine-tune function.
if "epochs" in train_kwargs:
    del train_kwargs["epochs"]
    
if "data" in train_kwargs:
    del train_kwargs["data"]

Although we specify 500 epochs, the process will automatically stop when performance stops improving:

In [ ]:
# Fine tune the YOLO model with the combined dataset.
model = fine_tune(
    model,
    os.path.join(IMAGES_PATH, COMBINED_DATASET_DIR, "dataset.yaml"),
    epochs=500,
    patience=25,
    **train_kwargs
)

# Export

In [ ]:
model.save(MODEL_OUTPUT_PATH)

# Evaluation

In [ ]:
model = YOLO(MODEL_OUTPUT_PATH)

In [ ]:
results = model.val(data=os.path.join(IMAGES_PATH, COMBINED_DATASET_DIR, "dataset.yaml"), split="test")

In [ ]:
def print_evaluation_results(results):
    print("Per-class metrics:")
    print(f"  Class indices: {results.ap_class_index}")
    print(f"  Precision for each class: {results.box.p}")
    print(f"  Recall for each class: {results.box.r}")
    print(f"  F1 score for each class: {results.box.f1}")

    # Mean results
    print(f"Mean precision: {results.box.mp}")
    print(f"Mean recall: {results.box.mr}")
    
    # Mean average precision (mAP)
    print(f"Mean average precision at IoU=0.50 to 0.95 (mAP50-95): {results.box.map}")
    print(f"Mean average precision at IoU=0.50 (mAP50): {results.box.map50}")
    print(f"Mean average precision at IoU=0.75 (mAP75): {results.box.map75}")

In [ ]:
print_evaluation_results(results)